In [1]:
import os
import pandas as pd
from IPython.display import display, Markdown
from tqdm.auto import tqdm

from common.consts import RESULTS_DIR, EVAL_SIZE
from common.utils import filename_to_obj, remove_index

/home/tdolega/miniconda/envs/p311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def results_as_pandas(filename):
    path = os.path.join(RESULTS_DIR, filename)
    data = pd.read_json(path, lines=True)

    if len(data) == 0:
        print(f"empty file: {filename}")
        return data

    params = filename_to_obj(filename)
    for k, v in params.items():
        data[k] = v

    data = data.explode("evaluations")
    data = data.rename_axis("question_idx").reset_index()

    data = pd.concat([data, data["evaluations"].apply(pd.Series)], axis=1)
    evaluation_keys = data["evaluations"].apply(pd.Series).columns
    for col in evaluation_keys:
        data = pd.concat([data, data[col].apply(pd.Series).add_prefix(f"{col}/")], axis=1)
        data = data.drop(columns=col)
    data = data.drop(columns=["evaluations"])

    return data


files = os.listdir(RESULTS_DIR)
params_names = list(filename_to_obj(files[0]).keys())
all_results = pd.concat([results_as_pandas(f) for f in tqdm(files)])
all_results.head()

100%|██████████| 41/41 [02:00<00:00,  2.94s/it]


,question_idx,question_id,llm,prompt_id,temperature,nli,ellm,sim,citations/ais_recall,citations/ais_precision,...,citations/supported,citations/citations,citations/correct_citations,citations/out_of_range,correctness/answer_overlap,correctness/answer_entail,correctness/citations_recall,correctness/citations_precision,quality/answer_relevance,quality/new_question
0,0,5abab42e55429955dce3eed2,Mistral-7B-Instruct-v0.2,1,0.8,t5_xxl_true_nli_mixture,Mistral-7B-Instruct-v0.2,all-MiniLM-L6-v2,0.250000,1.000,...,"[0, 0, 0, 1]","[[], [], [3], [4]]","[[], [], [True], [True]]","[0, 0, 0, 0]",1.0,1.0,1.0,1.0,0.649022,Which of P.K. Subban's brothers was drafted fi...
1,0,5abab42e55429955dce3eed2,Mistral-7B-Instruct-v0.2,1,0.8,t5_xxl_true_nli_mixture,Mistral-7B-Instruct-v0.2,all-MiniLM-L6-v2,0.083333,0.875,...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[[], [], [3], [4], [6], [], [], [], [], [4, 5]...","[[], [], [True], [True], [True], [], [], [], [...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1.0,0.0,1.0,0.5,0.757470,"Who are the two professional hockey players, b..."
2,0,5abab42e55429955dce3eed2,Mistral-7B-Instruct-v0.2,1,0.8,t5_xxl_true_nli_mixture,Mistral-7B-Instruct-v0.2,all-MiniLM-L6-v2,0.000000,1.000,...,"[0, 0, 0, 0, 0, 0]","[[], [], [4], [6], [], []]","[[], [], [True], [True], [], []]","[0, 0, 0, 0, 0, 0]",0.5,0.0,0.5,0.5,0.770844,"Who are the two professional hockey players, b..."
3,1,5a761900554299109176e648,Mistral-7B-Instruct-v0.2,1,0.8,t5_xxl_true_nli_mixture,Mistral-7B-Instruct-v0.2,all-MiniLM-L6-v2,1.000000,1.000,...,[1],[[1]],[[True]],[0],1.0,1.0,0.5,1.0,0.450042,What is the name of the intelligence firm that...
4,1,5a761900554299109176e648,Mistral-7B-Instruct-v0.2,1,0.8,t5_xxl_true_nli_mixture,Mistral-7B-Instruct-v0.2,all-MiniLM-L6-v2,1.000000,1.000,...,"[1, 1]","[[0, 1], [0, 1]]","[[True, True], [True, True]]","[0, 0]",1.0,1.0,1.0,1.0,0.772082,What is the name of the lobbying group founded...


In [3]:
all_obj_cols = all_results.select_dtypes(include=["object"]).columns
drop_obj_cols = list(set(all_obj_cols) - set(params_names))
drop_obj_cols.remove("question_id")
print(f"Dropping columns: {drop_obj_cols}")
all_num_results = all_results.drop(columns=drop_obj_cols)

Dropping columns: ['citations/sentences', 'citations/citations', 'quality/new_question', 'citations/correct_citations', 'citations/out_of_range', 'citations/supported']


In [4]:
eval_split = all_num_results[all_num_results["question_idx"] < EVAL_SIZE]
train_split = all_num_results[all_num_results["question_idx"] >= EVAL_SIZE]

In [5]:
AGG_FUNC = "max"


def aggregate(split):
    split = split.drop(columns=["question_idx"])
    results_with_std_for_each_question = split.groupby([*params_names, "question_id"]).agg([AGG_FUNC, "std"])
    results_for_each_model = results_with_std_for_each_question.groupby(params_names)
    results = results_for_each_model.mean()
    results["n_questions"] = results_for_each_model.size()
    return results


eval_results = aggregate(eval_split)
train_results = aggregate(train_split)

if eval_results["n_questions"].nunique() != 1:
    print("Warning: not all rows in evaluation have the same number of examples")

In [6]:
display(Markdown("### Prompts comparison"))
parameter_results = eval_results[eval_results.index.get_level_values("llm") == "Mistral-7B-Instruct-v0.2"]


def show_cleaned_results(short_eval_display, keep_index_name=None):
    short_eval_display = short_eval_display.copy()
    for index_name in ["temperature", "nli", "ellm", "sim", "prompt_id"]:
        if index_name == keep_index_name:
            continue
        short_eval_display = remove_index(short_eval_display, index_name)
    important_columns = ["citations/ais_recall", "citations/ais_precision", "correctness/answer_overlap", "correctness/answer_entail", "correctness/citations_recall", "correctness/citations_precision", "quality/answer_relevance"]
    short_eval_display = short_eval_display[important_columns]
    return short_eval_display


show_cleaned_results(parameter_results[parameter_results.index.get_level_values("temperature") == "0.1"], keep_index_name="prompt_id")

### Prompts comparison

citations/ais_recall            \
                                                    max       std   
llm                      prompt_id                                  
Mistral-7B-Instruct-v0.2 1                     0.606238  0.036560   
                         2                     0.559952  0.066146   
                         3                     0.621500  0.045033   
                         4                     0.550000  0.050999   

                                   citations/ais_precision            \
                                                       max       std   
llm                      prompt_id                                     
Mistral-7B-Instruct-v0.2 1                        0.901424  0.038350   
                         2                        0.887500  0.034978   
                         3                        0.913500  0.024787   
                         4                        0.864286  0.044519   

                                   correctness/answer_overlap            \
                                                          max       std   
llm                      prompt_id                                        
Mistral-7B-Instruct-v0.2 1                           0.883923  0.014915   
                         2                           0.877269  0.027905   
                         3                           0.869269  0.017321   
                         4                           0.862603  0.024249   

                                   correctness/answer_entail            \
                                                         max       std   
llm                      prompt_id                                       
Mistral-7B-Instruct-v0.2 1                              0.90  0.017321   
                         2                              0.88  0.023094   
                         3                              0.87  0.011547   
                         4                              0.85  0.011547   

                                   correctness/citations_recall            \
                                                            max       std   
llm                      prompt_id                                          
Mistral-7B-Instruct-v0.2 1                                0.775  0.025981   
                         2                                0.800  0.043301   
                         3                                0.690  0.034641   
                         4                                0.625  0.031754   

                                   correctness/citations_precision            \
                                                               max       std   
llm                      prompt_id                                             
Mistral-7B-Instruct-v0.2 1                                0.762833  0.050125   
                         2                                0.705500  0.046655   
                         3                                0.690167  0.042370   
                         4                                0.719024  0.036335   

                                   quality/answer_relevance            
                                                        max       std  
llm                      prompt_id                                     
Mistral-7B-Instruct-v0.2 1                         0.743680  0.028340  
                         2                         0.729335  0.030174  
                         3                         0.681964  0.027177  
                         4                         0.701345  0.026870

In [7]:
display(Markdown("### Temperature comparison"))
show_cleaned_results(parameter_results[parameter_results.index.get_level_values("prompt_id") == "1"], keep_index_name="temperature")

### Temperature comparison

citations/ais_recall            \
                                                      max       std   
llm                      temperature                                  
Mistral-7B-Instruct-v0.2 0.01                    0.597643  0.027143   
                         0.1                     0.606238  0.036560   
                         0.2                     0.614048  0.081991   
                         0.3                     0.638214  0.084729   
                         0.4                     0.717048  0.150051   
                         0.5                     0.693750  0.144597   
                         0.6                     0.667405  0.168079   
                         0.7                     0.712095  0.182757   
                         0.8                     0.693667  0.195393   
                         0.9                     0.750833  0.204617   
                         1.0                     0.730298  0.202885   
                         1.5                     0.725000  0.221513   
                         2.0                     0.702500  0.259358   
                         2.5                     0.538024  0.262768   

                                     citations/ais_precision            \
                                                         max       std   
llm                      temperature                                     
Mistral-7B-Instruct-v0.2 0.01                       0.901955  0.016922   
                         0.1                        0.901424  0.038350   
                         0.2                        0.918591  0.050804   
                         0.3                        0.914333  0.066155   
                         0.4                        0.963996  0.108934   
                         0.5                        0.942603  0.085049   
                         0.6                        0.941042  0.110804   
                         0.7                        0.941048  0.115458   
                         0.8                        0.959667  0.131938   
                         0.9                        0.968500  0.127270   
                         1.0                        0.968500  0.139181   
                         1.5                        0.976333  0.156983   
                         2.0                        0.963143  0.220605   
                         2.5                        0.922635  0.240513   

                                     correctness/answer_overlap            \
                                                            max       std   
llm                      temperature                                        
Mistral-7B-Instruct-v0.2 0.01                          0.870590  0.012990   
                         0.1                           0.883923  0.014915   
                         0.2                           0.887192  0.029793   
                         0.3                           0.908923  0.045707   
                         0.4                           0.908923  0.044607   
                         0.5                           0.910923  0.073768   
                         0.6                           0.886359  0.049038   
                         0.7                           0.905590  0.047054   
                         0.8                           0.914192  0.062564   
                         0.9                           0.892859  0.074434   
                         1.0                           0.884769  0.066776   
                         1.5                           0.903282  0.074745   
                         2.0                           0.878385  0.089997   
                         2.5                           0.876949  0.186147   

                                     correctness/answer_entail            \
                                                           max       std   
llm                      temperature                                       
Mistral-7B-Instruct-v0.2 0.01   

In [8]:
display(Markdown("### Evaluation results"))
eval_not_mistral = eval_results[eval_results.index.get_level_values("llm") != "Mistral-7B-Instruct-v0.2"]
eval_mistral = eval_results[(eval_results.index.get_level_values("llm") == "Mistral-7B-Instruct-v0.2") & (eval_results.index.get_level_values("prompt_id") == "1") & (eval_results.index.get_level_values("temperature") == "0.1")]
eval_display = pd.concat([eval_not_mistral, eval_mistral])
eval_display = eval_display[eval_display.index.get_level_values("llm") != "Mixtral-8x7B-Instruct-v0.1.Q8_0"]  # use this model only for training data, because we already have same model without quantization
eval_display = remove_index(eval_display, "prompt_id")
eval_display = eval_display.sort_values(by=("correctness/citations_recall", AGG_FUNC), ascending=False)
eval_display

### Evaluation results

prompt_id  \
                                                                                                                     
llm                        temperature nli                     ellm                     sim                          
rag-tge_Mistral_v2-4480    0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
rag-tge_Llama-3-8B_v1      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
rag-tge_Mistral_v2-3360    0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
rag-tge_Llama-3-8B_v2      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         0   
rag-tge_Llama-3-8B_v3      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         4   
rag-tge_Mistral_v6         0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         4   
rag-tge_Mistral.Q8         0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
gpt-3.5-turbo-0125         0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
zephyr-orpo-141b-A35b-v0.1 0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
Mixtral-8x7B-Instruct-v0.1 0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
gpt-4-turbo                0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
Meta-Llama-3-70B-Instruct  0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
qwen1_5-110b-chat          0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
Meta-Llama-3-8B-Instruct   0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
Mistral-7B-Instruct-v0.2   0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
zephyr-7b-beta             0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
qwen1_5-14b-chat-q8_0      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
qwen1_5-32b-chat-q8_0      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
c4ai-command-r-plus        0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
qwen1_5-7b-chat-q8_0       0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
gemma-1.1-7b-it            0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
Phi-3-mini-4k-instruct     0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
gemma-1.1-2b-it            0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
Mistral-7B-Instruct-v0.1   0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   

                                                                                                         citations/ais_recall  \
                                                                                                                          max   
llm                        temperature nli                     ellm                     sim                                     
rag-tge_Mistral_v2-4480    0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.855570   
rag-tge_Llama-3-8B_v1      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.734354   
rag-tge_Mistral_v2-3360    0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.813959   
rag-tge_Llama-3-8B_v2      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.769950   
rag-tge_Llama-3

In [9]:
clean_results = show_cleaned_results(eval_display)
for agg in [AGG_FUNC, "std"]:
    display(Markdown(f"# {agg}"))
    for sort_by in [
        ("citations/ais_recall", agg),
        ("citations/ais_precision", agg),
        ("correctness/answer_overlap", agg),
        ("correctness/answer_entail", agg),
        ("correctness/citations_recall", agg),
        ("correctness/citations_precision", agg),
        ("quality/answer_relevance", agg),
    ]:
        display(Markdown(f"### sorted by: {sort_by[0]}"))
        results = clean_results.sort_values(by=sort_by, ascending=False)
        results = results[[col for col in results.columns if col[1] == agg]]
        display(results)

# max

### sorted by: citations/ais_recall

,citations/ais_recall,citations/ais_precision,correctness/answer_overlap,correctness/answer_entail,correctness/citations_recall,correctness/citations_precision,quality/answer_relevance
,max,max,max,max,max,max,max
llm,,,,,,,
rag-tge_Llama-3-8B_v3,0.875667,0.975167,0.895744,0.92,0.955,0.955333,0.790450
rag-tge_Mistral.Q8,0.864167,0.974167,0.870923,0.87,0.930,0.936190,0.779305
rag-tge_Mistral_v2-4480,0.855570,0.979417,0.934590,0.91,0.995,0.933333,0.780416
rag-tge_Mistral_v6,0.854167,0.970000,0.931256,0.95,0.955,0.976667,0.796144
rag-tge_Mistral_v2-3360,0.813959,0.984115,0.961667,0.92,0.990,0.918333,0.782177
gpt-3.5-turbo-0125,0.801167,0.997500,0.895910,0.92,0.875,0.962333,0.787486
gpt-4-turbo,0.775000,0.990000,0.885892,0.91,0.820,0.949333,0.777481
rag-tge_Llama-3-8B_v2,0.769950,0.984613,0.949359,0.97,0.985,0.877024,0.808191


### sorted by: citations/ais_precision

,citations/ais_recall,citations/ais_precision,correctness/answer_overlap,correctness/answer_entail,correctness/citations_recall,correctness/citations_precision,quality/answer_relevance
,max,max,max,max,max,max,max
llm,,,,,,,
gpt-3.5-turbo-0125,0.801167,0.997500,0.895910,0.92,0.875,0.962333,0.787486
qwen1_5-32b-chat-q8_0,0.534167,0.996667,0.880392,0.90,0.660,0.961667,0.731615
qwen1_5-110b-chat,0.607500,0.993333,0.888982,0.91,0.780,0.947333,0.787043
gpt-4-turbo,0.775000,0.990000,0.885892,0.91,0.820,0.949333,0.777481
rag-tge_Llama-3-8B_v2,0.769950,0.984613,0.949359,0.97,0.985,0.877024,0.808191
rag-tge_Mistral_v2-3360,0.813959,0.984115,0.961667,0.92,0.990,0.918333,0.782177
qwen1_5-14b-chat-q8_0,0.676667,0.983333,0.880154,0.87,0.690,0.931667,0.754467
rag-tge_Mistral_v2-4480,0.855570,0.979417,0.934590,0.91,0.995,0.933333,0.780416


...

,citations/ais_recall,citations/ais_precision,correctness/answer_overlap,correctness/answer_entail,correctness/citations_recall,correctness/citations_precision,quality/answer_relevance
,std,std,std,std,std,std,std
llm,,,,,,,
zephyr-orpo-141b-A35b-v0.1,0.134222,0.101052,0.060333,0.040415,0.092376,0.108477,0.059428
gemma-1.1-7b-it,0.122999,0.082995,0.075822,0.075056,0.080829,0.101471,0.043209
zephyr-7b-beta,0.090558,0.068665,0.028674,0.046188,0.080056,0.088494,0.041852
qwen1_5-7b-chat-q8_0,0.063671,0.057158,0.024118,0.023094,0.074282,0.078327,0.034364
qwen1_5-110b-chat,0.095486,0.026943,0.020207,0.017321,0.051962,0.023286,0.055224
Phi-3-mini-4k-instruct,0.030503,0.069580,0.012317,0.023094,0.051188,0.064275,0.037250
c4ai-command-r-plus,0.060382,0.061399,0.023094,0.000000,0.043868,0.054568,0.046987
Mixtral-8x7B-Instruct-v0.1,0.061199,0.029692,0.001443,0.011547,0.034641,0.037528,0.030875


### sorted by: correctness/citations_precision

,citations/ais_recall,citations/ais_precision,correctness/answer_overlap,correctness/answer_entail,correctness/citations_recall,correctness/citations_precision,quality/answer_relevance
,std,std,std,std,std,std,std
llm,,,,,,,
rag-tge_Llama-3-8B_v2,0.120419,0.049620,0.015031,0.040415,0.005774,0.135618,0.066080
zephyr-orpo-141b-A35b-v0.1,0.134222,0.101052,0.060333,0.040415,0.092376,0.108477,0.059428
gemma-1.1-7b-it,0.122999,0.082995,0.075822,0.075056,0.080829,0.101471,0.043209
rag-tge_Llama-3-8B_v1,0.100349,0.030020,0.013723,0.023094,0.008660,0.098810,0.071202
zephyr-7b-beta,0.090558,0.068665,0.028674,0.046188,0.080056,0.088494,0.041852
rag-tge_Mistral_v2-3360,0.146211,0.032981,0.030777,0.011547,0.025981,0.084337,0.063220
qwen1_5-7b-chat-q8_0,0.063671,0.057158,0.024118,0.023094,0.074282,0.078327,0.034364
rag-tge_Mistral_v2-4480,0.134646,0.031338,0.017918,0.023094,0.025981,0.070338,0.055383


### sorted by: quality/answer_relevance

,citations/ais_recall,citations/ais_precision,correctness/answer_overlap,correctness/answer_entail,correctness/citations_recall,correctness/citations_precision,quality/answer_relevance
,std,std,std,std,std,std,std
llm,,,,,,,
rag-tge_Llama-3-8B_v1,0.100349,0.030020,0.013723,0.023094,0.008660,0.098810,0.071202
rag-tge_Llama-3-8B_v2,0.120419,0.049620,0.015031,0.040415,0.005774,0.135618,0.066080
rag-tge_Mistral_v2-3360,0.146211,0.032981,0.030777,0.011547,0.025981,0.084337,0.063220
zephyr-orpo-141b-A35b-v0.1,0.134222,0.101052,0.060333,0.040415,0.092376,0.108477,0.059428
rag-tge_Mistral_v2-4480,0.134646,0.031338,0.017918,0.023094,0.025981,0.070338,0.055383
qwen1_5-110b-chat,0.095486,0.026943,0.020207,0.017321,0.051962,0.023286,0.055224
c4ai-command-r-plus,0.060382,0.061399,0.023094,0.000000,0.043868,0.054568,0.046987
gemma-1.1-7b-it,0.122999,0.082995,0.075822,0.075056,0.080829,0.101471,0.043209


In [10]:
display(Markdown("### Training results"))
train_display = remove_index(train_results, "prompt_id")
train_display

### Training results

prompt_id  \
                                                                                                                          
llm                             temperature nli                     ellm                     sim                          
Meta-Llama-3-70B-Instruct       0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
Mixtral-8x7B-Instruct-v0.1      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
Mixtral-8x7B-Instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
qwen1_5-110b-chat               0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
qwen1_5-32b-chat-q8_0           0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
zephyr-orpo-141b-A35b-v0.1      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   

                                                                                                              citations/ais_recall  \
                                                                                                                               max   
llm                             temperature nli                     ellm                     sim                                     
Meta-Llama-3-70B-Instruct       0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.768950   
Mixtral-8x7B-Instruct-v0.1      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.734613   
Mixtral-8x7B-Instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.684853   
qwen1_5-110b-chat               0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.669000   
qwen1_5-32b-chat-q8_0           0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.582407   
zephyr-orpo-141b-A35b-v0.1      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.563212   

                                                                                                                         \
                                                                                                                    std   
llm                             temperature nli                     ellm                     sim                          
Meta-Llama-3-70B-Instruct       0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.031242   
Mixtral-8x7B-Instruct-v0.1      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.063516   
Mixtral-8x7B-Instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.070232   
qwen1_5-110b-chat               0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.117130   
qwen1_5-32b-chat-q8_0           0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.031801   
zephyr-orpo-141b-A35b-v0.1      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.127016   

                                                                                                              citations/ais_precision  \
                                                                                                                                  max   
llm                             temperature nli                     ellm                     sim                                        
Meta-Llama-3-70B-Instruct       0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2                0.967017   
Mixtral-8x7B-Instruct-v0.1      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2                0.956884   
Mi